# numerics 

> numerics module for numpy and pandas related data processing 

In [ ]:
#| default_exp numerics

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import math
from datetime import datetime
from functools import reduce
from typing import Dict, List, Optional, Tuple, Union, cast
from zoneinfo import ZoneInfo
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
#| export
def assemble_state_ser(
    state_columns: pd.DataFrame, tz: ZoneInfo
) -> Tuple[pd.Series, int]:
    """
    assemble state df from state_columns dataframe
    order is vital for the model:
    "timestep, velocity, thrust, brake"
    contiguous storage in each measurement
    due to sort_index, output:
    [col0: brake, col1: thrust, col2: timestep, col3: velocity]
    return:
    state: pd.Series
    table_row_start: int
    """

    # state_columns['timestep'] = pd.to_datetime(datetime.now().timestamp(), unit='us').tz_localize(tz)
    state: pd.Series = cast(
        pd.Series,
        (state_columns.stack().swaplevel(0, 1)),
    )
    state.name = "state"
    state.index.names = ["rows", "idx"]
    state.sort_index(
        inplace=True
    )  # sort by rows and idx (brake, thrust, timestep, velocity)
    # str_as_type = f"datetime64[us,{tz.key}]"  # type: ignore
    # state['timestep'].astype(str_as_type, copy=False)

    vel_stats = state["velocity"].astype("float").describe()

    # 0~20km/h; 7~30km/h; 10~40km/h; 20~50km/h; ...
    # average concept
    # 10; 18; 25; 35; 45; 55; 65; 75; 85; 95; 105
    #   13; 18; 22; 27; 32; 37; 42; 47; 52; 57; 62;
    # here upper bound rule adopted
    if vel_stats["max"] < 20:
        table_row_start = 0
    elif vel_stats["max"] < 30:
        table_row_start = 1
    elif vel_stats["max"] < 120:
        table_row_start = math.floor((vel_stats["max"] - 30) / 10) + 2
    else:
        table_row_start = 16  # cycle higher than 120km/h!
    # get the row of the table

    return state, table_row_start

In [ ]:
#| hide
from data_io_nbdev.utils import generate_eos_df, generate_state

In [ ]:
#| hide
df = generate_eos_df()
df["state"]

In [ ]:
#| hide
state = df['state'][["timestep", "velocity", "thrust", "brake"]].iloc[0]
# state = df['state']["timestep"]
# state["timestep"].values
state = pd.DataFrame([state["timestep"].values, state["velocity"].values, state["thrust"].values, state["brake"].values]).T
state.columns = ["timestep", "velocity", "thrust", "brake"]
state

In [ ]:
#| hide
state_ser = state.stack().swaplevel(0, 1)
state_ser.name = "state"
state_ser.index.names = ["rows", "idx"]
state_ser.sort_index(inplace=True)
state_ser

In [ ]:
#| hide
tz = ZoneInfo("Asia/Shanghai")
# state = df['state'].stack
ser_state, row_start = assemble_state_ser(state, tz)

In [ ]:
#| hide
assemble_state_ser(state, tz)[0]

In [ ]:
assert assemble_state_ser(state, tz)[1] == 0

In [ ]:
assert isinstance(assemble_state_ser(state, tz)[0], pd.Series) == True

In [ ]:
from fastcore.test import *

In [ ]:
test_eq(isinstance(assemble_state_ser(state, tz)[0], pd.Series), True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()